In [9]:
from web3 import Web3
from abi import UNISWAP_FACTORY_ABI, UNISWAP_PAIR_ABI, ERC20_ABI
from itertools import combinations

In [10]:

# Configuração da rede Polygon
# polygon_rpc_url = "https://polygon-rpc.com"
# web3 = Web3(Web3.HTTPProvider(polygon_rpc_url))

polygon_rpc_url = "https://polygon-mainnet.infura.io/v3/079d35a3c196469191c9accc1d68d25b"
web3 = Web3(Web3.HTTPProvider(polygon_rpc_url))

# Verificando conexão
if not web3.provider.is_connected():
    raise ConnectionError("Falha ao conectar à rede Polygon")

print("Conectado à rede Polygon")

# Endereços das fábricas das DEXs
DEX_FACTORIES = {
    "QuickSwap": web3.to_checksum_address("0x5757371414417b8c6caad45baef941abc7d3ab32"),
    "SushiSwap": web3.to_checksum_address("0xc35dadb65012ec5796536bd9864ed8773abc74c4")
}

# Exemplo de uso do endereço checksum
print("Endereços no formato checksum:")
for name, address in DEX_FACTORIES.items():
    print(f"{name}: {address}")



Conectado à rede Polygon
Endereços no formato checksum:
QuickSwap: 0x5757371414417b8C6CAad45bAeF941aBc7d3Ab32
SushiSwap: 0xc35DADB65012eC5796536bD9864eD8773aBc74C4


In [62]:


# Função para obter reservas da QuickSwap
def get_token_reserves(factory_address, token_a, token_b):
    try:
        factory = web3.eth.contract(address=factory_address, abi=UNISWAP_FACTORY_ABI)
        pair_address = factory.functions.getPair(token_a, token_b).call()

        if pair_address == "0x0000000000000000000000000000000000000000":
            print(f"Par inexistente para {token_a} e {token_b}.")
            return None
    
        pair = web3.eth.contract(address=pair_address, abi=UNISWAP_PAIR_ABI)
        reserves = pair.functions.getReserves().call()
    
        token0 = pair.functions.token0().call()
        token1 = pair.functions.token1().call()
    
        decimals_token0 = get_decimals(token0)
        decimals_token1 = get_decimals(token1)
    
        if token0.lower() == token_a.lower():
            return (reserves[0] / (10 ** decimals_token0), reserves[1] / (10 ** decimals_token1))
        else:
            return (reserves[1] / (10 ** decimals_token1), reserves[0] / (10 ** decimals_token0))
    except Exception as e:
        print(f"Erro ao consultar o par: {e}")
        return None


# Função para consultar preços na API do SushiSwap
def get_sushi_prices():
    url = "https://api.sushi.com/price/v1/137"
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Falha ao obter preços da API do SushiSwap.")

# Função para calcular arbitragem com logs detalhados
def calculate_arbitrage(factory_quick, token_a, token_b, investment, fee=0.003):
    # Obter reservas da QuickSwap
    reserves_quick = get_token_reserves(factory_quick, token_a, token_b)
    if not reserves_quick:
        return "Reservas não encontradas para QuickSwap."

    # Obter preços da SushiSwap
    sushi_prices = get_sushi_prices()
    price_a_sushi = sushi_prices.get(token_a)
    price_b_sushi = sushi_prices.get(token_b)

    if not price_a_sushi or not price_b_sushi:
        return "Preços não encontrados para SushiSwap."

    # Calcular razões de preço
    price_ratio_quick = reserves_quick[1] / reserves_quick[0]  # TOKEN_B / TOKEN_A
    price_ratio_sushi = price_a_sushi / price_b_sushi          # TOKEN_A / TOKEN_B

    # Logs informativos
    print(f"\n[INFO] Reservas na QuickSwap: {reserves_quick[0]:.6f} TOKEN_A, {reserves_quick[1]:.6f} TOKEN_B")
    print(f"[INFO] Preços na SushiSwap: {price_a_sushi:.6f} USD/TOKEN_A, {price_b_sushi:.6f} USD/TOKEN_B")
    print(f"[INFO] Razão QuickSwap: {price_ratio_quick:.6f} TOKEN_B/TOKEN_A")
    print(f"[INFO] Razão SushiSwap: {price_ratio_sushi:.6f} TOKEN_B/TOKEN_A")

    # Simulação de swaps
    if price_ratio_sushi > price_ratio_quick:
        # Compre na QuickSwap e venda na SushiSwap
        token_a_received = (investment / price_ratio_quick) * (1 - fee)  # TOKEN_A recebido na QuickSwap
        token_b_received = token_a_received * price_ratio_sushi * (1 - fee)  # TOKEN_B recebido na SushiSwap

        
        print(f"\n[SWAP] Compre na QuickSwap: {investment:.2f} TOKEN_B → {token_a_received:.6f} TOKEN_A")
        print(f"[SWAP] Venda na SushiSwap: {token_a_received:.6f} TOKEN_A → {token_b_received:.2f} TOKEN_B")

        profit = token_b_received - investment
        return f"[RESULTADO] Arbitragem: Lucro de {profit:.2f} TOKEN_B."
    elif price_ratio_quick > price_ratio_sushi:
        # Compre na SushiSwap e venda na QuickSwap
        token_a_received = (investment / price_ratio_sushi) * (1 - fee)  # TOKEN_A recebido na SushiSwap
        token_b_received = token_a_received * price_ratio_quick * (1 - fee)  # TOKEN_B recebido na QuickSwap

        print(f"\n[SWAP] Compre na SushiSwap: {investment:.2f} TOKEN_B → {token_a_received:.6f} TOKEN_A")
        print(f"[SWAP] Venda na QuickSwap: {token_a_received:.6f} TOKEN_A → {token_b_received:.2f} TOKEN_B")

        profit = token_b_received - investment
        return f"[RESULTADO] Arbitragem: Lucro de {profit:.2f} TOKEN_B."
    else:
        return "[RESULTADO] Sem oportunidade de arbitragem."



In [63]:

# Endereços e exemplo de uso
factory_quick = web3.to_checksum_address("0x5757371414417b8c6caad45baef941abc7d3ab32")  # QuickSwap
factory_uniswap = web3.to_checksum_address("0x1F98431c8aD98523631AE4a59f267346ea31F984")  # UniSwap



In [64]:



# token_avee = web3.to_checksum_address("0xD6DF932A45C0f255f85145f286eA0b292B21C90B")  # Aave
# token_usdc = web3.to_checksum_address("0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174")  # USDC

# token_weth = web3.to_checksum_address("0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619")

# token_wbtc = web3.to_checksum_address("0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6")
# token_wmatic = web3.to_checksum_address("0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270")
# token_link = web3.to_checksum_address("0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39")
# token_bet = web3.to_checksum_address("0xbF7970D56a150cD0b60BD08388A4A75a27777777")
# token_ldo = web3.to_checksum_address("0xC3C7d422809852031b44ab29EEC9F1EfF2A58756")
# token_falcon = web3.to_checksum_address("0x45b0bF770edAd9b3A8Aaa015B47a1f46a7cF956F")


# investment = 1000  # Investimento em USDC

# # Calcular arbitragem
# result = calculate_arbitrage(factory_uni, token_avee, token_usdc, investment)
# print(result)


In [67]:
from itertools import product

# Lista de tokens
tokens = [
    {"symbol": "AAVE", "address": web3.to_checksum_address("0xD6DF932A45C0f255f85145f286eA0b292B21C90B")},
    {"symbol": "WETH", "address": web3.to_checksum_address("0x7ceB23fD6bC0adD59E62ac25578270cFf1b9f619")},
    {"symbol": "WBTC", "address": web3.to_checksum_address("0x1BFD67037B42Cf73acF2047067bd4F2C47D9BfD6")},
    {"symbol": "WMATIC", "address": web3.to_checksum_address("0x0d500B1d8E8eF31E21C99d1Db9A6444d3ADf1270")},
    {"symbol": "LINK", "address": web3.to_checksum_address("0x53E0bca35eC356BD5ddDFebbD1Fc0fD03FaBad39")},
    {"symbol": "BET", "address": web3.to_checksum_address("0xbF7970D56a150cD0b60BD08388A4A75a27777777")},
    {"symbol": "LDO", "address": web3.to_checksum_address("0xC3C7d422809852031b44ab29EEC9F1EfF2A58756")},
    # {"symbol": "FALCON", "address": web3.to_checksum_address("0x45b0bF770edAd9b3A8Aaa015B47a1f46a7cF956F")},
]

# USDC token
token_usdc = {"symbol": "USDC", "address": web3.to_checksum_address("0x2791Bca1f2de4661ED88A30C99A7a9449Aa84174")}

# Gerar combinações com USDC
pairs = [(token_usdc, token) for token in tokens]

investment = 1000  # Investimento em USDC


# Loop para testar arbitragem com todas as combinações
for pair in pairs:
    token_a = pair[0]
    token_b = pair[1]
    
    print(f"\n[TESTE] Arbitragem entre {token_a['symbol']} e {token_b['symbol']}")
    result = calculate_arbitrage(factory_quick, token_b["address"], token_a["address"], investment)
    print(result)



[TESTE] Arbitragem entre USDC e AAVE
<web3._utils.datatypes.Contract object at 0x7f6f9aaf9580> 0x7c76B6B3FE14831A39C0fec908DA5f17180df677

[INFO] Reservas na QuickSwap: 4.634022 TOKEN_A, 1353.979801 TOKEN_B
[INFO] Preços na SushiSwap: 292.922239 USD/TOKEN_A, 1.000486 USD/TOKEN_B
[INFO] Razão QuickSwap: 292.182451 TOKEN_B/TOKEN_A
[INFO] Razão SushiSwap: 292.779999 TOKEN_B/TOKEN_A

[SWAP] Compre na QuickSwap: 1000.00 TOKEN_B → 3.412251 TOKEN_A
[SWAP] Venda na SushiSwap: 3.412251 TOKEN_A → 996.04 TOKEN_B
[RESULTADO] Arbitragem: Lucro de -3.96 TOKEN_B.

[TESTE] Arbitragem entre USDC e WETH
<web3._utils.datatypes.Contract object at 0x7f6f9b76b740> 0x853Ee4b2A13f8a742d64C8F088bE7bA2131f670d

[INFO] Reservas na QuickSwap: 380.511963 TOKEN_A, 1225306.506511 TOKEN_B
[INFO] Preços na SushiSwap: 3220.576761 USD/TOKEN_A, 1.000491 USD/TOKEN_B
[INFO] Razão QuickSwap: 3220.152385 TOKEN_B/TOKEN_A
[INFO] Razão SushiSwap: 3218.996168 TOKEN_B/TOKEN_A

[SWAP] Compre na SushiSwap: 1000.00 TOKEN_B → 0.3097

In [73]:
def sort_tokens(token_a, token_b):
    if token_a.lower() > token_b.lower():
        return token_b, token_a
    return token_a, token_b


import hashlib



# Loop para testar arbitragem com todas as combinações
for pair in pairs:
    token_a = pair[0]
    token_b = pair[1]
    
    print(f"\n[TESTE] Arbitragem entre {token_a['symbol']} e {token_b['symbol']}")
    print(result)

    factory = web3.eth.contract(address=factory_uniswap, abi=UNISWAP_FACTORY_ABI)
    
    # Ordenar tokens conforme a lógica do contrato
    token_a, token_b = sort_tokens(token_a["address"], token_b["address"])

    pair_address = factory.functions.getPair(token_b, token_a).call()
    


[TESTE] Arbitragem entre USDC e AAVE
[RESULTADO] Arbitragem: Lucro de 91.04 TOKEN_B.


ContractLogicError: ('execution reverted', 'no data')